In [33]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
from datetime import datetime
from os import getcwd
import pickle

import backtrader as bt
import pandas as pd

from polaristools.polarisbot import PolarisBot
from strategies import mystrategies

In [34]:
polaris = PolarisBot()

# Define functions

In [35]:
def optimization(
                symbol:str,
                timeframe:str,
                cash:int,
                sizer:int,
                comm:float,
                sample:dict,
                custom_strategy:mystrategies,
                parameters:dict
                ):
    cerebro = bt.Cerebro()
    
    cerebro.broker.set_cash( cash )
    cerebro.addsizer(bt.sizers.PercentSizer, percents=sizer)
    cerebro.broker.setcommission(commission=comm, mult=parameters.get('leverage_factor'))
    
    if timeframe == '1d':
        tframe = bt.TimeFrame.Days
    else:
        tframe = bt.TimeFrame.Minutes
    
    filename = f'df_klines_{symbol}_{timeframe}'
    df = polaris.dataframeFromBinary(filename)
    if sample:
        df = df.loc[sample.get('start'):sample.get('end')].copy()
    
    feed = bt.feeds.PandasData(dataname=df, timeframe=tframe, compression = int(timeframe[:-1]))
    cerebro.adddata(feed)
    
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='tradeanalyzer')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    
    cerebro.optstrategy(custom_strategy, **parameters)
    backtests = cerebro.run()
    
    return backtests

In [36]:
def parse_analyzers(backtests):
    try:
        all_params = [dict(zip(
                list(x[0].params._getkeys()),
                list(x[0].params._getvalues()),
            ))for x in backtests]
        df_params = pd.DataFrame(all_params)
        df_params.drop(columns=['verbose'], inplace=True)
    except:
        print('SOMETHING HAS FAILED WHILE TRYING TO RETRIEVE THE PARAMETERS FROM STRATEGY.', __main__)
        return
    try:
        trades = [dict(
            pnl_net         = x[0].analyzers.tradeanalyzer.get_analysis()['pnl']['net']['total'], 
            trades          = x[0].analyzers.tradeanalyzer.get_analysis()['total']['total'], 
            won             = x[0].analyzers.tradeanalyzer.get_analysis()['won']['total'], 
            lost            = x[0].analyzers.tradeanalyzer.get_analysis()['lost']['total'],
            
            long_won        = x[0].analyzers.tradeanalyzer.get_analysis()['long']['won'],
            short_won       = x[0].analyzers.tradeanalyzer.get_analysis()['short']['won'],
            
            long_lost       = x[0].analyzers.tradeanalyzer.get_analysis()['long']['lost'],
            short_lost      = x[0].analyzers.tradeanalyzer.get_analysis()['short']['lost'],
            
            longs_pnl       = x[0].analyzers.tradeanalyzer.get_analysis()['long']['pnl']['total'],
            shorts_pnl      = x[0].analyzers.tradeanalyzer.get_analysis()['short']['pnl']['total'],
            
            moneydown_max   = x[0].analyzers.drawdown.get_analysis()['max']['moneydown'],
            )for x in backtests
        ]
        df_trades = pd.DataFrame(trades)
        df_results = pd.concat([df_params, df_trades], axis=1)
        return df_results
    except:
        print('FAILED PARSING ANALYZERS')
        return pd.DataFrame()
        # return df_params
        

In [37]:
def filter_results(dataframe, symbol:str, timeframe:str, by_col:str):
    if dataframe.empty:
        print(f'FAILED TEST ON symbol: {symbol}, tf: {timeframe}')
        return dataframe
    labels = ['pnl_net','won','lost',
            'trades','moneydown_max',
            'long_won','long_lost',
            'short_won','short_lost',
            'longs_pnl','shorts_pnl',
            'leverage_factor','aroon_timeperiod','ema',]
    best5 = dataframe[labels].nlargest(5, by_col)
    worse3 = dataframe[labels].nsmallest(3, by_col)
    
    bs = f'{symbol} {timeframe} BEST_5' 
    ws = f'{symbol} {timeframe} WORSE_3'
    best_horse = pd.concat([best5,worse3], keys=[bs, ws], axis=0)
    
    return best_horse

In [46]:
def dataframeToBinary(dataframe:pd.DataFrame, name:str):
    filename = f'logs/OPT-{name}.pckl'
    with open(filename, 'wb') as bin_df:
        pickle.dump(dataframe, bin_df)
    print(f'{filename} persisted as binary ok')

In [57]:
def dataframeToBinary(dataframe:pd.DataFrame, name:str):
    filename = f'logs/OPT-{name}.pckl'
    with open(filename, 'wb') as bin_df:
        pickle.dump(dataframe, bin_df)
    print(f'{filename} persisted as binary ok')
    
def dataframeFromBinary(name:str):
    filename = f'logs/OPT-{name}.pckl'
    try:
        df_bin = open(filename, 'rb')
        dataframe = pickle.load(df_bin)
        df_bin.close()
        return dataframe
    except:
        print(f'Filename: {filename} does not exists yet.', __name__)
        return None

# LOOP #20 - RUN Multiple tests

In [39]:
ema                 = list(range(50, 101, 10))
aroon_timeperiod    = list(range(50, 101, 10))

In [40]:
ema

[50, 60, 70, 80, 90, 100]

In [41]:
aroon_timeperiod

[50, 60, 70, 80, 90, 100]

In [42]:
[x+25 for x in ema]

[75, 85, 95, 105, 115, 125]

In [43]:
# def loop_optimizations(symbols:list, timeframes:list, sample:dict, parameters:dict):

symbols    = ['BTCUSDT','BNBUSDT', 'DOGEUSDT', 'ETHUSDT']   #len 4
timeframes = ['240m', '120m', '60m', '30m', '15m']          #len 5
sample     = {'start':'2021-01-01', 'end':'2022-08-12'}

# parameters = dict(
#                     enter_long          = True,
#                     enter_short         = True,
                    
#                     ema                 = list(range(50, 101, 10)),
#                     aroon_timeperiod    = list(range(50, 101, 10)),
#                     leverage_factor     = 1.0,
# )

# Store each results Dataframe here.
symbols_df = pd.DataFrame()
idx20 = 0
total_it = int(len(symbols)*len(timeframes))
for symbol in symbols:
    ema = list(range(50, 101, 10))
    aroon_timeperiod = list(range(50, 101, 10))
    for timeframe in timeframes:
        parameters = dict(
            enter_long          = True,
            enter_short         = True,
            
            ema                 = ema,
            aroon_timeperiod    = aroon_timeperiod,
            leverage_factor     = 1.0,
        )
        k=25
        ema = [x+k for x in ema]
        aroon_timeperiod = [x+k for x in aroon_timeperiod]
        backtest = optimization(
            symbol          = symbol,
            timeframe       = timeframe,
            cash            = 100,
            sizer           = 25,
            comm            = 0.05,
            sample          = sample,
            custom_strategy = mystrategies.AroonPlusMa,
            parameters      = parameters,
        )
        df = parse_analyzers(backtest)
        
        best5_worse3 = filter_results(dataframe=df, symbol=symbol, timeframe=timeframe, by_col='pnl_net')
        
        if symbols_df.empty:
            # First iteration check.
            symbols_df = best5_worse3
        else:
            symbols_df = pd.concat([symbols_df, best5_worse3], axis=0)
        
        idx20+=1
        now = datetime.now()
        print(f'Iteration #{idx20} of {total_it}. At {now}. SYMBOL: {symbol} TIMEFRAME: {timeframe}')
        # display(best5_worse3)

Iteration #1 of 9. At 2022-08-15 21:59:20.239450. SYMBOL: BTCUSDT TIMEFRAME: 240m
Iteration #2 of 9. At 2022-08-15 21:59:44.652832. SYMBOL: BTCUSDT TIMEFRAME: 120m
Iteration #3 of 9. At 2022-08-15 22:00:37.423645. SYMBOL: BTCUSDT TIMEFRAME: 60m
Iteration #4 of 9. At 2022-08-15 22:02:42.553334. SYMBOL: BTCUSDT TIMEFRAME: 30m
Iteration #5 of 9. At 2022-08-15 22:05:58.351520. SYMBOL: BTCUSDT TIMEFRAME: 15m
FAILED PARSING ANALYZERS
FAILED TEST ON symbol: BNBUSDT, tf: 240m
Iteration #6 of 9. At 2022-08-15 22:06:11.981670. SYMBOL: BNBUSDT TIMEFRAME: 240m
Iteration #7 of 9. At 2022-08-15 22:06:36.773248. SYMBOL: BNBUSDT TIMEFRAME: 120m
Iteration #8 of 9. At 2022-08-15 22:07:25.786194. SYMBOL: BNBUSDT TIMEFRAME: 60m
Iteration #9 of 9. At 2022-08-15 22:09:19.468928. SYMBOL: BNBUSDT TIMEFRAME: 30m
Iteration #10 of 9. At 2022-08-15 22:13:04.003624. SYMBOL: BNBUSDT TIMEFRAME: 15m
Iteration #11 of 9. At 2022-08-15 22:13:21.178686. SYMBOL: DOGEUSDT TIMEFRAME: 240m
Iteration #12 of 9. At 2022-08-15 2

## Persist results.

In [44]:
fname = 'best_results_aroon_2'

dataframeToBinary(symbols_df, fname)

best_results_opt persisted as binary ok


In [ ]:
# df_x = dataframeFromBinary('best_results_opt')
# df_x.head()

In [56]:
# symbols_df.sort_values(by=['pnl_net'], ascending=False).head(20)

pnl_net  won  lost  trades  moneydown_max  \
DOGEUSDT 60m BEST_5  35  103.537686    2     1       3     117.350454   
                     29  103.537686    2     1       3     117.350454   
                     23  103.537686    2     1       3     117.350454   
                     17  103.537686    2     1       3     117.350454   
                     11  103.537686    2     1       3     117.350454   
DOGEUSDT 120m BEST_5 0    81.320387    2     2       4     106.307263   
DOGEUSDT 240m BEST_5 30   72.413255    1     2       4     434.025249   
                     24   72.413255    1     2       4     434.025249   
                     12   72.413255    1     2       4     434.025249   
                     18   72.413255    1     2       4     434.025249   
BNBUSDT 60m BEST_5   9    72.378276    1     3       4      73.865857   
                     3    72.378276    1     3       4      73.865857   
                     15   72.378276    1     3       4      73.865857   
                     21   72.378276    1     3       4      73.865857   
                     27   72.378276    1     3       4      73.865857   
DOGEUSDT 120m BEST_5 30   58.171770    1     3       4     108.641883   
                     12   56.445806    1     2       3     106.307263   
                     6    56.445806    1     2       3     106.307263   
                     18   54.772795    1     3       4     106.307263   
DOGEUSDT 15m BEST_5  9    40.651556    3     6       9     158.103312   

                         long_won  long_lost  short_won  short_lost  \
DOGEUSDT 60m BEST_5  35         1          1          1           0   
                     29         1          1          1           0   
                     23         1          1          1           0   
                     17         1          1          1           0   
                     11         1          1          1           0   
DOGEUSDT 120m BEST_5 0          1          2          1           0   
DOGEUSDT 240m BEST_5 30         1          1          0           1   
                     24         1          1          0           1   
                     12         1          1          0           1   
                     18         1          1          0           1   
BNBUSDT 60m BEST_5   9          1          0          0           3   
                     3          1          0          0           3   
                     15         1          0          0           3   
                     21         1          0          0           3   
                     27         1          0          0           3   
DOGEUSDT 120m BEST_5 30         1          2          0           1   
                     12         1          2          0           0   
                     6          1          2          0           0   
                     18         1          2          0           1   
DOGEUSDT 15m BEST_5  9          2          4          1           2   

                          longs_pnl  shorts_pnl  leverage_factor  \
DOGEUSDT 60m BEST_5  35  101.010069    2.527617              1.0   
                     29  101.010069    2.527617              1.0   
                     23  101.010069    2.527617              1.0   
                     17  101.010069    2.527617              1.0   
                     11  101.010069    2.527617              1.0   
DOGEUSDT 120m BEST_5 0    79.857665    1.462723              1.0   
DOGEUSDT 240m BEST_5 30   78.470989   -6.057734              1.0   
                     24   78.470989   -6.057734              1.0   
                     12   78.470989   -6.057734              1.0   
                     18   78.470989   -6.057734              1.0   
BNBUSDT 60m BEST_5   9    91.735371  -19.357095              1.0   
                     3    91.735371  -19.357095              1.0   
                     15   91.735371  -19.357095              1.0   
                     21   91.735371  -19.357095   